In [ ]:
'https://storage.googleapis.com/chrome-for-testing-public/124.0.6367.60/mac-arm64/chrome-mac-arm64.zip'

In [2]:
import pandas as pd 
import requests

In [74]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [162]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [164]:
username_css= "#username"
password_css='#password'
button_css='#loginform > div > button'

In [ ]:
main_page_url = 'https://www-chicagomanualofstyle-org.libproxy2.usc.edu/book/ed17/frontmatter/toc.html'

await page.goto(main_page_url)
await page.locator(username_css).fill("spangher")
await page.locator(password_css).fill("Pica_pic0Pica_pic5")
await page.locator(button_css).click()

overview_content= await page.content()

In [53]:
with open('../corpora/chicago-style-guide/main.html', 'w') as f:
    f.write(overview_content)

In [166]:
soup = BeautifulSoup(overview_content)
chapter_lis = soup.find_all('li', attrs={'class': 'toc'})
chapter_hrefs = list(map(lambda x: x.find('a')['href'], chapter_lis))

In [70]:
chapter_endpoints = list(filter(lambda x: 'part2' in x, chapter_hrefs))

In [76]:
url_parts = urlparse(main_page_url)

In [83]:
domain = f'{url_parts.scheme}://{url_parts.netloc}'

In [94]:
all_subchapter_endpoints = []

In [ ]:
from tqdm.auto import tqdm
import time

for e in tqdm(chapter_endpoints):
    await page.goto(f'{domain}{e}')
    subchapter_html = await page.content()
    soup = BeautifulSoup(subchapter_html)
    subchapter_lis = soup.find_all('li', attrs={'class': 'ctoc2'})
    subchapter_endpoints = list(map(lambda x: x.find('a')['href'], subchapter_lis))
    all_subchapter_endpoints += subchapter_endpoints

In [99]:
len(all_subchapter_endpoints)

1093

In [102]:
with open('../corpora/chicago-style-guide/all-subchapter-urls.txt', 'w') as f:
    for e in all_subchapter_endpoints:
        f.write(e)
        f.write('\n')

In [167]:
all_style_guide_rules = []
for e in tqdm(all_subchapter_endpoints):
    await page.goto(f'{domain}{e}')
    page_content = await page.content()
    style_guide_item = BeautifulSoup(page_content)
    page_section = style_guide_item.find('div', attrs={'class': 'page-section'})
    
    section_number = page_section.find('span', attrs={'class': 'section-number'}).get_text()
    title = page_section.find('span', attrs={'class': 'section-title'}).get_text()
    
    trail = page_section.find('p', attrs={'class': 'trail'}).find_all('a')
    trail = list(map(lambda x: {'href': x['href'], 'text': x.get_text()}, trail))
    content = page_section.find('div', attrs={'class': 'section-content'}).get_text().strip()
    
    style_guide_packet = {
        'url': e,
        'section_number': section_number,
        'title': title,
        'hierarchy': trail, 
        'content': content
    }
    all_style_guide_rules.append(style_guide_packet)
    time.sleep(.5)

  0%|          | 0/1093 [00:00<?, ?it/s]

In [175]:
import pandas as pd 

In [185]:
all_style_guide_rules[-3]

{'url': '/book/ed17/part2/ch13/psec070.html',
 'section_number': '13.70',
 'title': 'Parenthetical source following a block quotation',
 'hierarchy': [{'href': '/book/ed17/part2/ch13/toc.html',
   'text': 'Chapter Contents'},
  {'href': '/book/ed17/part2/ch13/psec064.html',
   'text': 'Attributing Quotations in Text'},
  {'href': '/book/ed17/part2/ch13/psec070.html',
   'text': 'Sources Following Block Quotations and Poetry Extracts'}],
 'content': 'The source of a block quotation is given in parentheses at the end of the quotation and in the same type size. The opening parenthesis appears after the final punctuation mark of the quoted material. No period either precedes or follows the closing parenthesis. See also 6.101, 15.26.\nIf you happen to be fishing, and you get a strike, and whatever it is starts off with the preliminaries of a vigorous fight; and by and by, looking down over the side through the glassy water, you see a rosy golden gleam, the mere specter of a fish, shining be

In [177]:
all_style_guide_rules_df = pd.DataFrame(all_style_guide_rules)

In [179]:
all_style_guide_rules_df.to_json('../corpora/chicago-style-guide/all-rules.jsonl', lines=True, orient='records')

# Query LLMs for Training Data

In [180]:
## on endeavor

In [ ]:
from vllm import LLM, SamplingParams
pluslab_cache_dir='/mnt/data09/spangher/Projects/press-releases/models'
endeavor_cache_dir = "/project/jonmay_231/spangher/huggingface_cache"
model = "CohereForAI/c4ai-command-r-v01"
model = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = LLM(
    model=model, dtype='half',
    tensor_parallel_size=4, 
    download_dir=endeavor_cache_dir
)

In [193]:
style_guide_rule_title, style_guide_rule = (
    all_style_guide_rules_df
     .iloc[0][['title', 'content']]
)

In [243]:
# style_guide_rule_title= 'Noun gender'
# style_guide_rule = "English nouns have no true gender, as that property is understood in many other languages. For example, whether a noun refers to a masculine or feminine person or thing does not determine the form of the accompanying article as it does in French, German, Spanish, and many other languages. Still, some English words—almost exclusively nouns denoting people or animals—are inherently masculine {uncle} {rooster} {lad} or feminine {aunt} {hen} {lass} and take the gender-appropriate pronouns. But most English nouns are common in gender and may refer to either sex {relative} {chicken} {child}. Many words once considered strictly masculine—especially words associated with jobs and professions—have been accepted as common (or indefinite) in gender over time {author} {executor} {proprietor}. Similarly, many forms made feminine by the addition of a suffix {aviatrix} {poetess} have been essentially abandoned."
n = 20
prompt = """
You are a journalist's assistant. Here is a rule from the Chicago Manual of Style that either captures a grammatical inaccuracy, or a changing style:

The title of this rule is: "{style_guide_rule_title}"

The rule is:

```{style_guide_rule}```

If you are able to identify a clear rule or linguistic preference expressed, 
write me {n} examples of pairs of two sentences: one that violates the rule ("bad sentence") 
and one corrected version of the same sentence ("good sentence"). 
Make them topically sound like news-article sentences: they should be about current events, or similar news writing.
Respond with a list of {n} python dictionaries, each with the keys "bad sentence", "good sentence".

If you cannot identify a clear rule or linguistic preference expressed, say "No clear rule or preference expressed".

Don't say anything else.
"""

In [ ]:
sampling_params = SamplingParams(temperature=0.4, top_p=0.95, max_tokens=100)
outputs = llm.generate([prompt], sampling_params)

generated_text = outputs[0].outputs[0].text
generated_text

# Prompt OpenAI

In [208]:
from transformers import AutoTokenizer

In [210]:
tok = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=open('/Users/spangher/.openai-isi-key.txt').read().strip())

In [219]:
len(tok.encode(prompt)) * 2 * 1000

362000

In [259]:
all_training_data = []
for _, (url, style_guide_rule_title, style_guide_rule) in tqdm(
    all_style_guide_rules_df[['url', 'title', 'content']].iloc[100:].iterrows(), 
    total=len(all_style_guide_rules_df)
):
    prompt_str = prompt.format(style_guide_rule=style_guide_rule, style_guide_rule_title=style_guide_rule_title, n=n)    
    messages = [
        {
            "role": "system", 
            "content": "You are a journalist's assistant who helps me perform line-edits."
        }, {
            'role': 'user',
            'content': prompt_str,
        }
    ]
    break
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages
    )
    response = completion.choices[0].message.content

  0%|          | 0/1093 [00:00<?, ?it/s]

In [274]:
completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    temperature=.3
)
response = completion.choices[0].message.content

In [261]:
import ast

In [273]:
print(messages)

[{'role': 'system', 'content': "You are a journalist's assistant who helps me perform line-edits."}, {'role': 'user', 'content': '\nYou are a journalist\'s assistant. Here is a rule from the Chicago Manual of Style that either captures a grammatical inaccuracy, or a changing style:\n\nThe title of this rule is: "Linking verbs"\n\nThe rule is:\n\n```A linking verb (also called a copula or connecting verb) is one that links the subject to a closely related word in the predicate—a subjective complement. The linking verb itself does not take an object, because it expresses a state of being instead of an action {Mr. Block is the chief executive officer} {that snake is venomous} {his heart’s desire is to see his sister again}. There are two kinds of linking verbs: be-verbs and intransitive verbs that are used in a weakened sense, such as appear, become, feel, look, seem, smell, and taste. The weakened intransitive verbs often have a figurative sense akin to that of become, as in He fell heir

In [275]:
ast.literal_eval(response)

[{'bad sentence': 'The economy appears very shaky.',
  'good sentence': 'The economy appears to be very shaky.'},
 {'bad sentence': 'The new policy feels a bit rushed.',
  'good sentence': 'The new policy feels like it was a bit rushed.'},
 {'bad sentence': 'The lake smells foul.',
  'good sentence': 'The lake smells like something foul.'},
 {'bad sentence': 'The candidate looks the part of a leader.',
  'good sentence': 'The candidate looks like the part of a leader.'},
 {'bad sentence': 'The soup tastes too salty.',
  'good sentence': 'The soup tastes like it is too salty.'},
 {'bad sentence': 'The city seems quieter post-pandemic.',
  'good sentence': 'The city seems to be quieter post-pandemic.'},
 {'bad sentence': 'The painting looks a masterpiece.',
  'good sentence': 'The painting looks like a masterpiece.'},
 {'bad sentence': 'The agreement feels a significant step.',
  'good sentence': 'The agreement feels like a significant step.'},
 {'bad sentence': 'The new model appears a 

In [266]:
print(messages[1]['content'])


You are a journalist's assistant. Here is a rule from the Chicago Manual of Style that either captures a grammatical inaccuracy, or a changing style:

The title of this rule is: "Linking verbs"

The rule is:

```A linking verb (also called a copula or connecting verb) is one that links the subject to a closely related word in the predicate—a subjective complement. The linking verb itself does not take an object, because it expresses a state of being instead of an action {Mr. Block is the chief executive officer} {that snake is venomous} {his heart’s desire is to see his sister again}. There are two kinds of linking verbs: be-verbs and intransitive verbs that are used in a weakened sense, such as appear, become, feel, look, seem, smell, and taste. The weakened intransitive verbs often have a figurative sense akin to that of become, as in He fell heir to a large fortune (he didn’t physically fall on or into anything) or The river ran dry (a waterless river doesn’t run—it has dried up). 